In [2]:
import pandas as pd
import numpy as np

In [3]:
index50 = pd.read_csv('台灣50指數資料.csv', encoding='utf-16', sep='\t')

In [4]:
#計算市值與權重並新增欄位
index50['市值'] = index50['公眾流通係數'] * index50['前日調整收盤價']*index50['股數']
index50['成份股'] = index50['成份股'].str.split(' ').str[0]
index50

,證券代碼,年月日,成份股,指數因子,公眾流通係數,股數,前日調整收盤價,前日市值比重,市值
0,TWN50 台灣50指數,20190102,1101,0.0021,0.8700,5108059911,35.60,1.0261,1.582068e+11
1,TWN50 台灣50指數,20190102,1102,0.0011,0.6900,3361447198,33.95,0.5107,7.874358e+10
2,TWN50 台灣50指數,20190102,1216,0.0023,0.8500,5682015421,69.80,2.1865,3.371140e+11
3,TWN50 台灣50指數,20190102,1301,0.0023,0.7700,6365740781,101.00,3.2110,4.950637e+11
4,TWN50 台灣50指數,20190102,1303,0.0027,0.7200,7930821589,75.50,2.7962,4.311195e+11
...,...,...,...,...,...,...,...,...,...
48896,TWN50 台灣50指數,20221230,5880,0.0044,0.7200,14008935446,25.65,1.0845,2.587170e+11
48897,TWN50 台灣50指數,20221230,6415,0.0001,0.6600,381075821,432.50,0.4560,1.087781e+11
48898,TWN50 台灣50指數,20221230,6505,0.0006,0.1500,9525959652,79.70,0.4774,1.138828e+11
48899,TWN50 台灣50指數,20221230,8046,0.0001,0.3249,646165487,231.00,0.2033,4.849595e+10


In [5]:
stock_list = index50['成份股'].unique()
stock_list

array(['1101', '1102', '1216', '1301', '1303', '1326', '1402', '2002',
       '2105', '2301', '2303', '2308', '2317', '2327', '2330', '2354',
       '2357', '2382', '2395', '2408', '2409', '2412', '2454', '2474',
       '2633', '2801', '2823', '2880', '2881', '2882', '2883', '2884',
       '2885', '2886', '2887', '2890', '2891', '2892', '2912', '3008',
       '3045', '3481', '3711', '4904', '4938', '5871', '5876', '5880',
       '6505', '9904', '2207', '9910', '2888', '6669', '2379', '6415',
       '3034', '8046', '1590', '2603', '2609', '2615', '8454', '3037',
       '6770'], dtype=object)

In [6]:
#將dataframe依照年月日切割，存入dict, ex. data_dict['2019/01/02']可取出當天的資料
data_of_the_date = {}
for i in range(0, len(index50)):
    if index50['年月日'][i] not in data_of_the_date:
        data_of_the_date[index50['年月日'][i]] = index50.iloc[i:i+1]
    else:
        data_of_the_date[index50['年月日'][i]] = pd.concat([data_of_the_date[index50['年月日'][i]], index50.iloc[i:i+1]])

In [7]:
#算權重
def get_weight(daily_data):
    stock_and_weight = {}
    total_value = daily_data['市值'].sum()
    for i in range(len(daily_data)):
        stock_and_weight[daily_data['成份股'][i]] = daily_data['市值'][i]/total_value
    return stock_and_weight

# # 使用範例
# get_weight(data_of_the_date[20190102])

In [8]:
import datetime as dt
import calendar

def find_last_weekday(year):
    months = [2, 5, 8, 11]
    last_days = []
    for month in months:
        num_days = calendar.monthrange(year, month)[1]

        # 取得當月最後一天的日期
        last_day = dt.date(year, month, num_days)

        # 若最後一天是週末，往前推算直到非週末
        while last_day.weekday() >= 5:  # 5 和 6 分別代表週六和週日
            last_day -= dt.timedelta(days=1)
        last_days.append(str(last_day).replace('-', ''))

    return last_days

# # 使用範例
# year = 2023
# find_last_weekday(year)


def find_last_friday(year):
    months = [2, 5, 8, 11]  # 所有月份
    last_fridays = []
    for month in months:
        # 找到該月份的最後一天
        num_days = calendar.monthrange(year, month)[1]
        last_day = dt.date(year, month, num_days)

        # 從最後一天開始往前遍歷，直到找到星期五
        while last_day.weekday() != 0:  # 4代表星期五
            last_day -= dt.timedelta(days=1)
        last_fridays.append(str(last_day).replace('-', ''))

    return last_fridays

# 使用範例
year = 2023
print(find_last_friday(year))


['20230227', '20230529', '20230828', '20231127']


台50指每3.6.9.12月會調整一次成分股，調整依照前一個月(2,5,8,11)最後一個交易日的資料為參考來調整權重。

In [9]:
import datetime

def get_month(date_intstring):
    date = datetime.datetime.strptime(str(date_intstring).split('\n')[0].strip(), '%Y%m%d')
    return date.month

def convert_to_minguo_date(date_str):
    date_obj = datetime.datetime.strptime(str(date_str), '%Y%m%d')
    minguo_year = date_obj.year - 1911
    minguo_date_str = date_obj.strftime(f"{minguo_year}%m%d")
    return minguo_date_str

In [10]:
def convert_format_to_float(s):
    s_without_commas = s.replace(',', '')
    s_as_integer = float(s_without_commas)
    return s_as_integer


In [11]:
import requests
import pandas as pd
import time

def get_stock_data_for_year(year, stock_code):
    stock_data = []
    for year in range(year-1, year+1):
        for month in range(1, 13):
            # 构建URL
            date = f"{year}{month:02d}01"  # 以指定年份和月份构建日期
            url = f"https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date={date}&stockNo={str(stock_code)}"

            # 发送请求
            response = requests.get(url)
            data = response.json()

            # 解析数据
            if 'data' in data:
                stock_data += data['data']
    if stock_data:
        df = pd.DataFrame(stock_data, columns=['Date', 'Volume', 'Turnover', 'Open', 'High', 'Low', 'Close', 'Change', 'Transaction'])
        df['Date'] = df['Date'].str.replace('/', '')
        c = df['Close'] != '--'
        df = df[c]
        df = df.reset_index(drop=True)
        df['Close'] = df['Close'].apply(convert_format_to_float)
        df['DailyReturn'] = df['Close'].pct_change()
        df = df[['Date', 'Close', 'DailyReturn']]
        return df

# 示範用法
year = 2022  
stock_code = 2330  

get_stock_data_for_year(year, stock_code)


,Date,Close,DailyReturn
0,1100104,536.0,NaN
1,1100105,542.0,0.011194
2,1100106,549.0,0.012915
3,1100107,565.0,0.029144
4,1100108,580.0,0.026549
...,...,...,...
485,1111226,456.5,0.003297
486,1111227,457.0,0.001095
487,1111228,451.0,-0.013129
488,1111229,446.0,-0.011086


In [12]:
stockPriceReturn_dict = {}
for stock in stock_list:
    stockPriceReturn_dict[stock] = get_stock_data_for_year(2020, stock)